# Network Expansion and Topology Modification

Demonstrates how to programmatically modify system topology by tapping existing lines and splitting buses. This is essential for planning studies where new substations or interconnections are evaluated.

In [ ]:
from gridwb import GridWorkBench
from gridwb.grid.components import Bus, Branch
import os

case_path = os.environ.get("SAW_TEST_CASE", "case.pwb")
wb = GridWorkBench(case_path)

In [ ]:
# Tap an existing transmission line
branches = wb[Branch, ['BusNum', 'BusNum:1', 'LineCircuit']]
if not branches.empty:
    b = branches.iloc[0]
    new_bus_num = int(wb[Bus, 'BusNum'].max() + 100)
    
    wb.esa.TapTransmissionLine(
        [b['BusNum'], b['BusNum:1'], b['LineCircuit']],
        pos_along_line=50.0,
        new_bus_number=new_bus_num,
        new_bus_name="Tapped_Substation"
    )
    print(f"Tapped line at 50% to create Bus {new_bus_num}")

In [ ]:
# Split a bus
target_bus = 1
split_bus_num = int(wb[Bus, 'BusNum'].max() + 1)

wb.esa.SplitBus(
    target_bus, 
    new_bus_number=split_bus_num,
    insert_bus_tie_line=True,
    branch_device_type="Breaker"
)
print(f"Split Bus {target_bus} to create Bus {split_bus_num}")

In [ ]:
# Validate changes with power flow
wb.pflow()

if wb.io.esa.is_converged():
    print("Power flow converged successfully.")
    print(f"Total buses now in system: {len(wb[Bus, :])}")
else:
    print("Power flow failed after topology changes.")